<a href="https://colab.research.google.com/github/DimitriLeandro/DA2Group10/blob/main/final_project_part_i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analytics II - Final Project Part I

## Methodology:

- Feature scaling

- Feature importance using Gini Index

- Deletion of unnecessary features

- Correlation between features

For each classifier:

    - SFS analysis using standard hyperparameters
    
    - Gridsearch using the selected features

    - Analysis of classification metrics

- Fitting of the best model with all the training data

- Predictions to the test dataset

## Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
import json
import xgboost as xgb
from time import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.feature_selection import SequentialFeatureSelector 
from sklearn.preprocessing import StandardScaler, LabelEncoder

warnings.filterwarnings("ignore")
#!pip install -U kaleido

## Downloading and unziping datasets

In [2]:
!wget https://uni-muenster.sciebo.de/s/bmzyEnwSscZ0tam/download?path=%2F&files=train_set.csv
!unzip -qq /content/download?path=%2F

--2022-06-27 17:10:22--  https://uni-muenster.sciebo.de/s/bmzyEnwSscZ0tam/download?path=%2F
Resolving uni-muenster.sciebo.de (uni-muenster.sciebo.de)... 128.176.4.4
Connecting to uni-muenster.sciebo.de (uni-muenster.sciebo.de)|128.176.4.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘download?path=%2F.1’

download?path=%2F.1     [          <=>       ]  36.05M  17.0MB/s    in 2.1s    

2022-06-27 17:10:25 (17.0 MB/s) - ‘download?path=%2F.1’ saved [37799682]

replace task_1/test_set.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Mounting drive to save results

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Training dataset preprocessing

In [ ]:
df = pd.read_csv('/content/task_1/train_set.csv') # /home/dimi/Downloads/task_1_datasets/train_set.csv
y  = df['y']
X  = pd.DataFrame(
    data    = StandardScaler().fit_transform(df.drop('y', axis=1)),
    columns = df.columns.drop('y')
)

In [ ]:
df_label_counts = pd.DataFrame(
    data    = np.transpose(np.unique(y, return_counts=True)),
    columns = ['label', 'count'] 
).sort_values(
    by        = 'count',
    ascending = False
).reset_index(
    drop = True
)

df_label_counts['pct'] = 100*df_label_counts['count']/y.size

df_label_counts

## Feature Importance

In [ ]:
# rf = RandomForestClassifier(
#     n_estimators = 1000,
#     n_jobs       = -1,
#     max_samples  = 0.66
# )
# rf.fit(X, y)

In [ ]:
# df_feature_importances = pd.DataFrame(
#     data    = zip(df.columns.drop('y'), rf.feature_importances_),
#     columns = ['feature', 'importance'] 
# ).sort_values(
#     by        = 'importance',
#     ascending = False,
# ).reset_index(
#     drop = True
# )

# df_feature_importances.to_csv(
#     'results/task_1/feature_importance_final_project_part_i.csv', #'/content/drive/MyDrive/Colab Notebooks/feature_importance_final_project_part_i.csv', 
#     index = False
# )

In [ ]:
max_features = 30
df_feature_importances = pd.read_csv('results/task_1/feature_importance_final_project_part_i.csv')
fig = px.bar(
    data_frame = df_feature_importances[:max_features],
    x          = 'feature',
    y          = 'importance'
)
fig.update_xaxes(tickangle=90)
fig.write_image('results/task_1/feature_importance_final_project_part_i.pdf')
fig.show()

## Correlation between features

In [ ]:
fig, ax = plt.subplots(figsize=(18,14))
sb.heatmap(X[df_feature_importances.loc[:8, 'feature']].corr(), cmap="Blues", annot=True, linewidths=0.1)
fig.savefig('results/task_1/feature_correlations_final_project_part_i.pdf')
fig.show()

Features are not correlated: no need to remove correlated features to prevent overfitting!

## SFS to each classifier

OBS: no need for SFS when using Random Forests.

In [ ]:
n_jobs               = -1
cv                   = 10
verbose              = 3
n_features_to_select = 100
plot_step            = 10
scoring              = 'accuracy'

In [ ]:
def run_sfs(estimator, X, y, n_features_to_select, cv, scoring, n_jobs, sorted_features=None, plot_step=1):
    
    if sorted_features is None:
        sfs = SequentialFeatureSelector(
            estimator            = estimator,
            direction            = 'forward',
            n_features_to_select = n_features_to_select,
            n_jobs               = n_jobs,
            cv                   = cv,
            scoring              = scoring,
        )
        sfs.fit(X, y)
        sorted_features = sfs.get_feature_names_out()
    
    kf               = KFold(n_splits=cv, shuffle=True)
    accs             = []
    stds             = []
    n_features_range = np.arange(plot_step, n_features_to_select+1, plot_step)

    for n_features in n_features_range:
        print(n_features)
        features   = sorted_features[:n_features]
        inner_accs = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X.loc[train_index][features], X.loc[test_index][features]
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            clf = estimator
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            inner_accs.append(accuracy_score(y_test, y_pred))

        accs.append(np.mean(inner_accs))
        stds.append(np.std(inner_accs))

    fig = px.line(
        x       = n_features_range,
        y       = accs,
        error_y = stds,
        title   = 'Sequential Feature Selector: ' + estimator.__class__.__name__,
        labels  = dict(
            x = 'Number of features', 
            y = 'Accuracy'
        )
    )
    fig.update_xaxes(tickvals=n_features_range)
    fig.write_image('results/task_1/sfs_{}_final_project_part_i.pdf'.format(estimator.__class__.__name__))
    fig.show()
    
    np.savetxt(
        fname = 'results/task_1/sfs_{}_final_project_part_i.csv'.format(estimator.__class__.__name__), 
        X     = sorted_features, 
        fmt   = '%s'
    )

### LDA

In [ ]:
%%time
run_sfs(
    LinearDiscriminantAnalysis(),
    X[df_feature_importances.loc[:100, 'feature']],
    y,
    n_features_to_select,
    cv,
    scoring,
    n_jobs,
    df_feature_importances['feature'],
    plot_step
)

### KNN

In [ ]:
%%time
run_sfs(
    KNeighborsClassifier(n_jobs=n_jobs),
    X[df_feature_importances.loc[:100, 'feature']],
    y,
    n_features_to_select,
    cv,
    scoring,
    n_jobs,
    df_feature_importances['feature'],
    plot_step
)

### SVM

In [ ]:
%%time
run_sfs(
    SVC(),
    X[df_feature_importances.loc[:100, 'feature']],
    y,
    n_features_to_select,
    cv,
    scoring,
    n_jobs,
    df_feature_importances['feature'],
    plot_step
)

### LR

In [ ]:
%%time
run_sfs(
    LogisticRegression(n_jobs=n_jobs),
    X[df_feature_importances.loc[:100, 'feature']],
    y,
    n_features_to_select,
    cv,
    scoring,
    n_jobs,
    df_feature_importances['feature'],
    plot_step
)

## Gridsearch

In [ ]:
def run_gridsearch(estimator, X, y, param_grid, cv, scoring, n_jobs, verbose):
    gs = GridSearchCV(
        estimator  = estimator,
        n_jobs     = n_jobs,
        cv         = cv,
        verbose    = verbose,
        scoring    = scoring,
        param_grid = param_grid
    )
    gs.fit(X, y)
    results = {
        'Estimator': str(estimator.__class__.__name__),
        'Number of features': X.shape[1],
        'Best result': '{:.3f} +- {:.3f}'.format(
            float(gs.cv_results_['mean_test_score'][gs.best_index_]),
            float(gs.cv_results_['std_test_score'][gs.best_index_]),
        ),
        'Best hyperparameters': gs.best_params_
    }
    with open(
        file = 'results/task_1/gridsearch_{}_final_project_part_i.json'.format(estimator.__class__.__name__),
        mode = 'w'
    ) as file:
        json.dump(results, file, indent=4)
    print(results)
    return gs

### Random Forest

In [ ]:
%%time

param_grid = dict(
    n_estimators      = [1000],
    criterion         = ['gini'],
    max_depth         = [10, 11, 12],
    class_weight      = [None],
    max_samples       = [0.66, 0.75, 1]
)

run_gridsearch(
    RandomForestClassifier(), 
    X[df_feature_importances.loc[:100, 'feature']], 
    y, 
    param_grid, 
    cv, 
    scoring, 
    n_jobs, 
    verbose
)

### KNN

In [ ]:
%%time

param_grid = dict(
    n_neighbors = [5, 10, 20, 30], #large K prevents overfitting!
)

run_gridsearch(
    KNeighborsClassifier(), 
    X[df_feature_importances.loc[:60, 'feature']], 
    y, 
    param_grid, 
    cv, 
    scoring, 
    1, # use just one kernel with KNN, otherwise, memory fuuuuuull 
    verbose
)

### SVM

In [ ]:
%%time

param_grid = dict(
    kernel = ['linear', 'poly', 'rbf'],
    degree = [2, 3]
)

run_gridsearch(
    SVC(), 
    X[df_feature_importances.loc[:50, 'feature']], 
    y, 
    param_grid, 
    cv, 
    scoring, 
    n_jobs, 
    verbose
)

### LR

In [ ]:
%%time

param_grid = dict(
    solver   = ['saga'],
    penalty  = ['elasticnet'],
    C        = [0.5, 0.75, 1, 1.25, 1.5],
    l1_ratio = [0.01, 0.25, 0.5, 0.75, 0.99]
)

run_gridsearch(
    LogisticRegression(), 
    X[df_feature_importances.loc[:80, 'feature']], 
    y, 
    param_grid, 
    cv, 
    scoring, 
    n_jobs, 
    verbose
)

### LDA

In [ ]:
%%time

param_grid = dict()

run_gridsearch(
    LinearDiscriminantAnalysis(), 
    X[df_feature_importances.loc[:90, 'feature']], 
    y, 
    param_grid, 
    cv, 
    scoring, 
    n_jobs, 
    verbose
)

## XG Boost

In [5]:
# encoding y
encoder   = LabelEncoder()
y_encoder = encoder.fit_transform(y)

# random split to start with (true cross validation later)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoder, test_size=0.2, random_state=0)

# xgb data matrices
D_train = xgb.DMatrix(X_train, label=y_train)
D_test  = xgb.DMatrix(X_test, label=y_test)

### XGB gridsearch (without sklearn API)

The reason not to use sklearn API is to speed up the process by using GPU support

In [10]:
initial_params = {
    'num_parallel_tree': 10, # used for boosting random forest
    'max_depth':         5,  # maximum tree depth for base learners
    'eta':               0.25, # boosting learning rate 
    'subsample':         0.75, # subsample ratio of the training instance
    'colsample_bytree':  0.75, # subsample ratio of columns when constructing each tree
    'colsample_bylevel': 1, # subsample ratio of columns when constructing each tree
    'colsample_bynode':  1, # subsample ratio of columns when constructing each tree
    'gamma':             1,  # minimum loss reduction required to make a further partition on a leaf
    'alpha':             0,  # L1 regularization term on weights 
    'lambda':            1,  # L2 regularization term on weights
    'sampling_method':   'gradient_based', # select samples based in their errors (not uniform)
    'tree_method':       'gpu_hist',     # make it fast
    'predictor':         'gpu_predictor' # make it fast
}

grid_params = {
    'max_depth':         [3, 5, 8, 10, 12, 15], # maximum tree depth for base learners
    'gamma':             [0, 1, 5, 10],   # minimum loss reduction required to make a further partition on a leaf
    'alpha':             [0, 0.5, 1, 1.5, 2], # L1 regularization term on weights 
    'lambda':            [0, 0.5, 1, 1.5, 2], # L2 regularization term on weights
    'subsample':         [0.66, 0.75, 1], # subsample ratio of the training instance
    'colsample_bytree':  [0.66, 0.75, 1], # subsample ratio of columns when constructing each tree
    'colsample_bylevel': [0.66, 0.75, 1], # subsample ratio of columns when constructing each tree
    'colsample_bynode':  [0.66, 0.75, 1], # subsample ratio of columns when constructing each tree
    'eta':               [0.1, 0.2, 0.25, 0.33],  # boosting learning rate 
}

In [11]:
best_params = initial_params.copy()
for param, values in grid_params.items():
  best_score = 9e9
  print('\nbest_params:', best_params)
  for value in values:
    print('testing {}={}'.format(param, value))
    xgb_params = best_params.copy()
    xgb_params.update({param: value})
    xgb_model = xgb.train(
        params                = xgb_params,
        dtrain                = D_train,
        evals                 = [(D_test,'eval')],
        num_boost_round       = 150,
        early_stopping_rounds = 5,
        verbose_eval          = 0,
    )
    if xgb_model.best_score < best_score:
      best_score = xgb_model.best_score
      best_params.update({param: value})
      print('new best value found: {}={} ({})'.format(param, value, best_score))


best_params: {'num_parallel_tree': 10, 'max_depth': 5, 'eta': 0.25, 'subsample': 0.75, 'colsample_bytree': 0.75, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'gamma': 1, 'alpha': 0, 'lambda': 1, 'sampling_method': 'gradient_based', 'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor'}
testing gamma=0
new best value found: gamma=0 (1.210185)
testing gamma=1
testing gamma=5
testing gamma=10

best_params: {'num_parallel_tree': 10, 'max_depth': 5, 'eta': 0.25, 'subsample': 0.75, 'colsample_bytree': 0.75, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'gamma': 0, 'alpha': 0, 'lambda': 1, 'sampling_method': 'gradient_based', 'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor'}
testing alpha=0
new best value found: alpha=0 (1.210185)
testing alpha=0.5
testing alpha=1
testing alpha=1.5
testing alpha=2

best_params: {'num_parallel_tree': 10, 'max_depth': 5, 'eta': 0.25, 'subsample': 0.75, 'colsample_bytree': 0.75, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'gamma': 0, 'alpha': 0, 

### Saving results and using CV to measure the accuracy

In [20]:
best_params.update({
    'num_parallel_tree': 25,              # incresing number of trees
    'num_class':         20,              # need this parameter if we want to use softmax
    'objective':         'multi:softmax', # for multiclass classification (use simple rmse to make it go faster)
})

best_params

{'alpha': 0,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.75,
 'eta': 0.3,
 'gamma': 0,
 'lambda': 0,
 'max_depth': 12,
 'num_class': 20,
 'num_parallel_tree': 25,
 'objective': 'multi:softmax',
 'predictor': 'gpu_predictor',
 'sampling_method': 'gradient_based',
 'subsample': 0.75,
 'tree_method': 'gpu_hist'}

In [15]:
with open('/content/drive/MyDrive/Colab Notebooks/xgb_hyperparameters.json', mode='w') as file:
  json.dump(best_params, file, indent=4)

In [21]:
xgb_model = xgb.train(
    params                = best_params,
    dtrain                = D_train,
    evals                 = [(D_train,'train'), (D_test,'eval')],
    num_boost_round       = 500,
    early_stopping_rounds = 10
)

[0]	train-merror:0.059714	eval-merror:0.134952
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
Stopping. Best iteration:
[15]	train-merror:0.000197	eval-merror:0.124074



0.8759262178779758

In [22]:
y_pred = xgb_model.predict(D_test, ntree_limit=xgb_model.best_ntree_limit).astype(np.int64)
accuracy_score(y_test, y_pred)

0.8759262178779758

### CV

In [24]:
D_train = xgb.DMatrix(X, label=y_encoder)

df_xgb_cv = xgb.cv(
    params                = best_params,
    dtrain                = D_train,
    num_boost_round       = 50,
    early_stopping_rounds = 5,
    nfold                 = 5,
    stratified            = True,
    verbose_eval          = True,
    shuffle               = True
)

In [27]:
1 - df_xgb_cv['test-merror-mean'].mean()

0.883711256